# LSTM - Long Short Term Memory

In [1]:
import os.path
import sys
import pickle

sys.path.append('../')
import numpy as np  # noqa: E402
from dl_framework.tensor import Tensor  # noqa: E402
from dl_framework.optimisers import SGD  # noqa: E402
from dl_framework.layers import Embedding, RNNCell, LSTMCell  # noqa: E402
from dl_framework.loss import CrossEntropy  # noqa: E402

## Data preprocessing

In [2]:
with open('../data/shakespeare/shakespeare.txt', 'r') as f:
    raw = f.read()
    vocab = list(set(raw))

word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for i, w in enumerate(vocab)}

indices = np.array([word2idx[w] for w in raw])

## Model with simple RNN cell

In [3]:
embed = Embedding(vocab_size=len(vocab), dim=512)
model = RNNCell(n_inputs=512, n_hidden=512, n_output=len(vocab))

criterion = CrossEntropy()
optimiser = SGD(parameters=model.get_parameters() + embed.get_parameters(), alpha=0.05)

batch_size = 32
bptt = 16
n_batches = int((indices.shape[0] / batch_size))

trimmed_indices = indices[:n_batches * batch_size]
batched_indices = trimmed_indices.reshape(batch_size, n_batches)
batched_indices = batched_indices.transpose()

input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:]

n_bptt = int(((n_batches - 1) / bptt))
input_batches = input_batched_indices[:n_bptt * bptt]
input_batches = input_batches.reshape(n_bptt, bptt, batch_size)
target_batches = target_batched_indices[:n_bptt * bptt]
target_batches = target_batches.reshape(n_bptt, bptt, batch_size)

In [4]:
def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = Tensor(np.array([word2idx[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
        output.data *= 10
        temp_dist = output.softmax()
        temp_dist /= temp_dist.sum()

        m = (temp_dist > np.random.rand()).argmax()
        #         m = output.data.argmax()
        c = vocab[m]
        input = Tensor(np.array([m]))
        s += c
    return s

In [5]:
def train(epochs=100):
    for e in range(epochs):
        total_loss = 0

        hidden = model.init_hidden(batch_size=batch_size)
        for batch_i in range(len(input_batches)):
            hidden = Tensor(hidden.data, autograd=True)
            loss = None
            losses = []
            for t in range(bptt):
                input = Tensor(input_batches[batch_i][t], autograd=True)
                rnn_input = embed.forward(input=input)
                output, hidden = model.forward(input=rnn_input, hidden=hidden)
                target = Tensor(target_batches[batch_i][t], autograd=True)
                batch_loss = criterion.forward(output, target)
                losses.append(batch_loss)
                if t == 0:
                    loss = batch_loss
                else:
                    loss = loss + batch_loss
            for loss in losses:
                ""
            loss.backward()
            optimiser.step()
            total_loss += loss.data
            log = '\r Iter: ' + str(e)
            log += ' - Batch ' + str(batch_i + 1) + '/' + str(n_batches)
            log += ' - Loss: ' + str(np.exp(total_loss / (batch_i + 1)))
            if batch_i == 0:
                log += ' - ' + generate_sample(70, '\n').replace('\n', ' ')
            if batch_i % 10 == 0 or batch_i - 1 == n_batches:
                print(log)
        optimiser.alpha *= 0.99
        print()

In [6]:
train_simple_rnn = False

if train_simple_rnn:
    train(epochs=100)

In [7]:
if train_simple_rnn:
    print(generate_sample(n=2000, init_char='\n'))

## Model with LSTM cell

In [8]:
embed = Embedding(vocab_size=len(vocab), dim=512)
model = LSTMCell(n_inputs=512, n_hidden=512, n_outputs=len(vocab))
model.w_ho.weight.data *= 0  # helps with training

criterion = CrossEntropy()
optimiser = SGD(parameters=model.get_parameters() + embed.get_parameters(), alpha=0.05)

batch_size = 16
bptt = 25
n_batches = int((indices.shape[0] / batch_size))

trimmed_indices = indices[:n_batches * batch_size]
batched_indices = trimmed_indices.reshape(batch_size, n_batches)
batched_indices = batched_indices.transpose()

input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:]

n_bptt = int(((n_batches - 1) / bptt))
input_batches = input_batched_indices[:n_bptt * bptt]
input_batches = input_batches.reshape(n_bptt, bptt, batch_size)
target_batches = target_batched_indices[:n_bptt * bptt]
target_batches = target_batches.reshape(n_bptt, bptt, batch_size)

In [9]:
def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = Tensor(np.array([word2idx[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
        output.data *= 15
        temp_dist = output.softmax()
        temp_dist /= temp_dist.sum()

#         m = (temp_dist > np.random.rand()).argmax() # sample from predictions
        m = output.data.argmax() # take the max prediction
        c = vocab[m]
        input = Tensor(np.array([m]))
        s += c
    return s

In [10]:
def train(epochs=100):
    min_loss = 1000
    for e in range(epochs):
        total_loss = 0
        n_loss = 0

        hidden = model.init_hidden(batch_size=batch_size)
        batches_to_train = len(input_batches)

        for batch_i in range(batches_to_train):
            hidden = (Tensor(hidden[0].data, autograd=True),
                      Tensor(hidden[1].data, autograd=True))
            losses = []
            
            for t in range(bptt):
                input = Tensor(input_batches[batch_i][t], autograd=True)
                rnn_input = embed.forward(input=input)
                output, hidden = model.forward(input=rnn_input, hidden=hidden)

                target = Tensor(target_batches[batch_i][t], autograd=True)
                batch_loss = criterion.forward(output, target)
                
                if t == 0:
                    losses.append(batch_loss)
                else:
                    losses.append(batch_loss + losses[-1])
            loss = losses[-1]
            
            loss.backward()
            optimiser.step()
            
            total_loss += loss.data / bptt
            epoch_loss = np.exp(total_loss / (batch_i + 1))
            if epoch_loss < min_loss:
                min_loss = epoch_loss
                # print(generate_sample(n=70, init_char='\n').replace('\n', ''))
            log = '\r Iter: ' + str(e)
            log += ' - Batch ' + str(batch_i + 1) + '/' + str(batches_to_train)
            log += ' - Min Loss: ' + str(min_loss)
            log += ' - Loss: ' + str(epoch_loss)
            if batch_i == 0:
                log += ' - ' + generate_sample(n=70, init_char='\n').replace('\n', ' ')
            if batch_i % 10 == 0 or batch_i - 1 == n_batches:
                print(log)
        optimiser.alpha *= 0.99            

In [11]:
pickle_path = '../models/lstm_model.pkl'

if not os.path.exists(pickle_path):
    train(epochs=100)

    with open(pickle_path, 'wb') as f:
        network_model = [model, embed, word2idx, idx2word]
        pickle.dump(model, f)
    
        
    print("Model trained and saved successfully")
        
else:
    with open(pickle_path, 'rb') as f:
        network_model = pickle.load(f)
        model = network_model[0]
        embed = network_model[1]
        word2idx = network_model[2]
        idx2word = network_model[3]
        vocab = list(word2idx.keys())
    
    print("Model loaded successfully")

Model loaded successfully


In [12]:
print(generate_sample(n=2000, init_char='\n'))

This is me them from the state.

But there is my lord, in the threrer: the did for what thou did grave the set precklersed say you the fair work, the sear'd in here, stranger the prerest thou will rere story for the set mistress.

First Must the set them from the set with the seem them for sun them from the serving strembert them from the persaked yourselves the did sled speak the straight the day, but of the set them for the pardof the sear'd from the dangery to make a forth the sear's the daughter, streat the set them servers of more for world so dridders the persery for that the presers, strain the present in the precorthin do no more from the set with his dead me in the season me the press the seasound to suds the dreas the set wear, the precory for the preceavers of my form,
As or the preconding up--burth the did strainess the dreat of my graighter, sicomportune shall we threat the daughter, silent in my hand I speak the set with the strangere, storn in the world inderer:
Nor I di